In [ ]:
"""
Erstellung von TIF-Dateien beliebiger Fernerkundungsindizes aus Sentinel-2-Daten 

Der Benutzer gibt seinen Dateipfad zum Sentinelordner und den gewünschten Index an 
(je nach Fragestellung, siehe unten)
Daraufhin geht das Programm in die Unterordner, wählt die Bänder in höchstmöglicher Auflösung aus und berechnet die Indizes.
Generelle Formel: (Rx-Ry)/(Rx+Ry)
Es können im Code beliebige Indices hinzugefügt werden, die sich aus den Sentinel-2-Bändern berechnen lassen! 

"""

In [5]:
"""
HIER NOCH NACH BELIEBEN MEHR INDIZES EINFÜGEN

NDWI = Fließ- und Stillgewässer, überflutete Bereiche 
NDVI = Vegetationsdichte und -intensität 
NDCI = Chlorophyll-Pigment von Pflanzen und Algen, Phytoplankton-Monitoring IN GEWÄSSERN (ökologische Gewässerqualität). Kein terrestrischer Index. 
NDMI = Vegetationsfeuchte
"""

index_dict = {
    "NDWI": ("03", "08"),   # (Grün, Nahinfrarot)
    "NDVI": ("08", "04"),    # (Nahinfrarot, Rot)
    "NDCI": ("04", "05") ,   # (Rot, Rot-Rand)
    "NDMI": ("8A", "11")     # (Vegetations-Rot-Rand, Kurzwelleninfrarot)
}

In [6]:
import os 
import numpy as np 
import pandas as pd 
import rasterio 
import matplotlib.pyplot as plt 
import tkinter as tk 
from tkinter import filedialog

In [ ]:
def select_folder(title="Ordner auswählen"):
    root = tk.Tk()
    root.withdraw()
    folder_selected = filedialog.askdirectory(title=title)
    return folder_selected


def find_band_files(s2_folder, band_numbers):
    """
    Sucht die passenden Sentinel-2-Bänder in der passenden Auflösung und vermeidet, dass Bänder mit unterschiedlicher Auflösung kombiniert werden.
    
    """
    
    resolution_paths = ["R10m", "R20m", "R60m"]
    band_files = {}
    resolutions_found = {}

    # erst alle möglichen auflösungen finden, dann aber nur mit bändern derselben auflösung weiterarbeiten 
    for root, dirs, files in os.walk(s2_folder):
        root = os.path.normpath(root)
        if any(res in root for res in resolution_paths):
            for file in files:
                for band in band_numbers:
                    if f"_B{band}_" in file and file.endswith(".jp2"):
                        res = os.path.basename(root)  # ordnername wird mit os.path abgerufen (zb "R10m")
                        if band not in resolutions_found:
                            resolutions_found[band] = []
                        resolutions_found[band].append(res)
                        break

    common_resolutions = set(resolutions_found[band_numbers[0]])
    for band in band_numbers[1:]:
        common_resolutions.intersection_update(resolutions_found[band])

    if not common_resolutions:
        print("Keine gemeinsame Auflösung für alle benötigten Bänder gefunden!")
        return {}

    resolution_order = {"R10m": 1, "R20m": 2, "R60m": 3}
    selected_resolution = min(common_resolutions, key=lambda x: resolution_order[x])

    # alle Bänder der gewählten Auflösung in band_files bringen
    for root, dirs, files in os.walk(s2_folder):
        root = os.path.normpath(root)
        if selected_resolution in root:
            for file in files:
                for band in band_numbers:
                    if f"_B{band}_" in file and file.endswith(".jp2"):
                        band_files[band] = os.path.normpath(os.path.join(root, file))
                        break

    return band_files

# Hier wird der Index basierend auf den gewählten Bändern berechnet
def calculate_index(band_paths, index):
    with rasterio.open(band_paths[index[0]]) as b1, rasterio.open(band_paths[index[1]]) as b2:
        band1 = b1.read(1).astype(np.float32)
        band2 = b2.read(1).astype(np.float32)
        index_result = np.where((band1 + band2) == 0, np.nan, (band1 - band2) / (band1 + band2))
        meta = b1.meta.copy()  # Meta erst nach dem Öffnen kopieren
        meta.update(dtype=rasterio.float32, count=1)
    return index_result, meta


# Der Index wird als Rasterdatei gespeichert, hierfür wird rasterio verwendet. 
# Das ganze wird aufgrund der Eigenschaften von .jp2 Dateien als GeoTIFF gespeichert. 
def save_raster(output_path, data, meta):
    meta.update(driver='GTiff')
    
    with rasterio.open(output_path, "w", **meta) as dst:
        dst.write(data, 1)


In [12]:
def main():
    s2_folder = select_folder("Sentinel-2 Ordner auswählen")
    output_folder = select_folder("Speicherort für Index wählen")
  
    print("Verfügbare Indizes:", list(index_dict.keys()))
    selected_index = input("Gewünschten Fernerkundungs-Index angeben: ")
    
    if selected_index not in index_dict:
        print("Ungültige Eingabe.")
        return
    
    bands_needed = index_dict[selected_index]
    band_paths = find_band_files(s2_folder, bands_needed)
    
    if len(band_paths) < 2:
        print("Nicht alle benötigten Bänder gefunden!")
        return
    
    index_data, meta = calculate_index(band_paths, bands_needed)
    folder_name = os.path.basename(os.path.normpath(s2_folder))
    folder_name_short = folder_name[:10]

    
    output_file = os.path.join(output_folder, f"{folder_name_short}_{selected_index}.tif")
    
    save_raster(output_file, index_data, meta)
    print(f"{selected_index}-TIF gespeichert unter {output_file}")

if __name__ == "__main__":
    main()

Verfügbare Indizes: ['NDWI', 'NDVI', 'NDCI', 'NDMI']


Gewünschten Fernerkundungs-Index angeben:  NDMI


C:\Users\Koch\AppData\Local\Temp\ipykernel_20188\1982445964.py:59: RuntimeWarning: invalid value encountered in divide
  index_result = np.where((band1 + band2) == 0, np.nan, (band1 - band2) / (band1 + band2))


NDMI-TIF gespeichert unter N:/DBU MONI/Fernerkundung/Sentinel_2_L2A_Daten/Cuxhaven_05.09.2024/S2B_MSIL2A_20240905T102559_N0511_R108_T32UME_20240905T163108.SAFE\Cuxhaven_0_NDMI.tif
